In [ ]:
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import theano.tensor as tt

import time

In [ ]:
# Set the number of unknown parameters (i.e. dimension of theta in posterior)
nparam = 3

# Number of draws from the distribution
ndraws = 1000

# Number of burn-in samples
nburn = 500

# Metropolis tuning parameters
tune = True
tune_interval = 100
discard_tuning = True

# Number of independent chains
nchains = 4

# Do blocked/compounds sampling in Metropolis and MLDA 
# Note: This choice applies only to the coarsest level in MLDA 
# (where a Metropolis sampler is used), all other levels use block sampling
blocked = True

# Set the sigma for inference
sigma = 0.01

# Sampling seed
sampling_seed = 12345

# Guess for prior - assume isotropic, homogeneous linear elasticity

E = np.exp(0.0)
nu = 0.3

G, lamb = E/(2*(1 + nu)), E * nu/((1 + nu)*(1 - 2 * nu))

In [ ]:
# Use a Theano Op along with the code within ./mlda to construct the likelihood

def my_loglik(my_model, theta, datapoints, data, sigma):
    output = model_wrapper(my_model, theta, datapoints)
    return - (0.5 / sigma ** 2) * np.sum((output - data) ** 2)

class LogLike(tt.Op):
    itypes = [tt.dvector]  # expects a vector of parameter values when called
    otypes = [tt.dscalar]  # outputs a single scalar value (the log likelihood)

    def __init__(self, my_model, loglike, data, x, sigma):

        # add inputs as class attributes
        self.my_model = my_model
        self.likelihood = loglike
        self.data = data
        self.x = x
        self.sigma = sigma

    def perform(self, node, inputs, outputs):
        theta = inputs  # this will contain my variables
        # call the log-likelihood function
        logl = self.likelihood(self.my_model, theta, self.x, self.data, self.sigma)
        outputs[0][0] = np.array(logl) # output the log-likelihood

In [ ]:
# Set up finest model and perform inference with PyMC3, using the MLDA algorithm
# and passing the coarse_models list created above.

traces = []
runtimes = []
acc = []
ess = []


with pm.Model():
    
    # Define priors on parameters
    param = [ None ] * 4
    
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
    
    param[0] = BoundedNormal('C10', mu = 0.5 * G, sigma = 1.0)
    param[1] = BoundedNormal('C01', mu = 0.0, sigma = 1.0)
    param[2] = BoundedNormal('C11', mu = 0.0, sigma = 1.0)
    param[3] = BoundedNormal('D1', mu = 0.5 * lamb, sigma = 1.0)

    # Convert m and c to a tensor vector
    theta = tt.as_tensor_variable(parameters)

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist('likelihood', lambda v: logl(v), observed={'v': theta})
    
    map_estimate = pm.find_MAP

    # Initialise a demetropolis step method.
    step_demetropolis = pm.DEMetropolis(tune='scaling', tune_interval=tune_interval)

    # Inference! 
    # DEMetropolis
    t_start = time.time()
    method_names.append("DEMetropolis")
    traces.append(pm.sample(draws=ndraws, step=step_demetropolis,
                            chains=nchains, tune=nburn,
                            discard_tuned_samples=discard_tuning,
                            random_seed=sampling_seed))
    runtimes.append(time.time() - t_start)